In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import os
import glob
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

## SVM（先頭30フレームの平均）

In [8]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_30_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.995516  0.584104  0.274504  0.461335  0.312321  0.185005   
question2     0.995715  0.642856  0.404782  0.516417  0.306051  0.173076   
question3     0.996076  0.649879  0.343308  0.467093  0.331270  0.158914   
question4     0.996124  0.614563  0.311653  0.455995  0.334674  0.176307   
question5     0.995153  0.623812  0.276110  0.575167  0.328001  0.308602   
question6     0.994046  0.619062  0.364727  0.511042  0.441243  0.228932   
question7     0.994499  0.646352  0.345298  0.471837  0.404303  0.205212   
question8     0.996331  0.577665  0.332649  0.453592  0.310819  0.294546   
question9     0.997603  0.716374  0.393647  0.480373  0.367472  0.221597   
question10    0.996228  0.737441  0.409118  0.466749  0.376113  0.197430   

                AU07      AU09      AU10      AU11  ... 

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.997032  0.829586  0.678959  0.473444  0.282765  0.150086   
question2     0.999617  0.824256  0.702520  0.618467  0.326531  0.067086   
question3     0.999493  0.812971  0.703231  0.533350  0.319321  0.075445   
question4     0.999578  0.817559  0.695245  0.528627  0.310379  0.079164   
question5     0.999319  0.794801  0.585748  0.525260  0.277647  0.089402   
question6     0.999069  0.778220  0.591008  0.578930  0.290599  0.085490   
question7     0.999455  0.721296  0.481096  0.504466  0.301816  0.066044   
question8     0.999409  0.803806  0.632477  0.404707  0.299120  0.069746   
question9     0.995739  0.865276  0.781643  0.373858  0.303077  0.075470   
question10    0.988750  0.863625  0.750207  0.665837  0.332409  0.079722   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label   

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.17589994 0.30066575 0.19992357 0.1440924  0.22858588 0.27126024
 0.2509433  0.20219742 0.30717126 0.3548264 ]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.07824664 0.0487162  0.02233599 0.0873238  0.06724508 0.10690727
 0.06047109 0.19467341 0.08157969 0.05895575]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.09113907 0.06219395 0.07265211 0.07102715 0.15556451 0.07341177
 0.07403974 0.12033848 0.030487  ]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'k

最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID70.csv
予測率: 44.44%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 1, 0, 0, 1, 0, 1, 1]
クラス1の確率: [0.15020718 0.42022221 0.65234781 0.24177138 0.12491131 0.66856775
 0.50654606 0.82776091 0.74422212]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID71.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.63039168 0.25658204 0.34536838 0.60146089 0.40100539 0.3336946
 0.43820663 0.37822901 0.42056764 0.26601085]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID72.csv
予測率: 55.56%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 1, 1, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.73443383 0.72932402 0.7186532  0.30441074 0.47400604 0.45119391
 0.45917382 0.562357   0.73103553]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID73.csv
予測

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.15336224 0.29908375 0.16593595 0.0970075  0.33364983 0.38325676
 0.23232196 0.26051962 0.38511222 0.40118911]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.04666453 0.02143365 0.03072039 0.03418718 0.02238859 0.03118756
 0.0209985  0.06055044 0.01725913 0.01246355]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.01598478 0.02015004 0.03834504 0.03936768 0.05716504 0.01823515
 0.03869502 0.08946388 0.00353344]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.10766575 0.14480318 0.19758215 0.1181085  0.11430011 0.013778

## SVM（先頭10フレームの平均）

In [11]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 平均を計算
    mean_values_per_label = first_10_frames_per_label.groupby("label").mean(numeric_only=True)
    
    # NaNを含む行を削除
    mean_values_per_label = mean_values_per_label.dropna(axis=0)
    
    # Yを追加
    mean_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, mean_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.994253  0.548917  0.309042  0.369101  0.340054  0.205853   
question2     0.995213  0.673667  0.509078  0.513324  0.264832  0.195403   
question3     0.996800  0.666825  0.336399  0.492604  0.332760  0.160899   
question4     0.996812  0.619893  0.317726  0.485552  0.308422  0.169937   
question5     0.995148  0.601538  0.277234  0.589133  0.330110  0.325815   
question6     0.995141  0.619397  0.362123  0.533383  0.458388  0.232431   
question7     0.994641  0.678903  0.357197  0.486914  0.381950  0.224525   
question8     0.997165  0.556887  0.305754  0.475889  0.310583  0.270285   
question9     0.998061  0.690526  0.395891  0.488248  0.373891  0.224995   
question10    0.996726  0.717515  0.432269  0.437019  0.448252  0.194634   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question2     0.999838  0.729022  0.419501  0.202886  0.418856  0.330912   
question3     0.999840  0.658654  0.426921  0.464003  0.268933  0.355504   
question4     0.999878  0.504299  0.278264  0.295859  0.354353  0.192616   
question5     0.999884  0.483455  0.380459  0.186780  0.447392  0.163884   
question6     0.999847  0.680622  0.406566  0.238349  0.327941  0.210354   
question7     0.998901  0.704993  0.249824  0.379479  0.296846  0.516543   
question8     0.999669  0.474076  0.178770  0.426425  0.270424  0.399522   
question9     0.999714  0.564358  0.264553  0.370031  0.339896  0.345312   
question10    0.999732  0.529260  0.165933  0.459368  0.301421  0.342639   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label                                       ...                             
question2

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.997739  0.444869  0.279833  0.596534  0.409172  0.233146   
question2     0.997044  0.360181  0.229466  0.708506  0.401036  0.168001   
question3     0.994291  0.460411  0.286429  0.521735  0.304848  0.258847   
question4     0.989487  0.568114  0.452583  0.399706  0.425778  0.265401   
question5     0.992415  0.601455  0.417037  0.456763  0.402993  0.349553   
question6     0.994112  0.533990  0.413505  0.563756  0.369665  0.306439   
question7     0.996506  0.698964  0.518986  0.538167  0.356811  0.174849   
question8     0.989735  0.613677  0.449690  0.446160  0.384153  0.335018   
question9     0.991478  0.572113  0.446910  0.485027  0.372616  0.403829   
question10    0.990900  0.527835  0.322207  0.685897  0.333907  0.387484   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.973208  0.694214  0.622000  0.408514  0.383721  0.360206   
question2     0.975367  0.619044  0.542112  0.450461  0.445346  0.302073   
question4     0.974625  0.564550  0.575708  0.270415  0.374156  0.215639   
question5     0.977450  0.591019  0.459954  0.419271  0.380544  0.627934   
question6     0.986087  0.564461  0.474480  0.470763  0.342777  0.447215   
question7     0.987884  0.365584  0.311886  0.757206  0.462605  0.131183   
question8     0.983288  0.522635  0.332969  0.581542  0.375834  0.362720   
question9     0.973260  0.526712  0.397457  0.604820  0.386238  0.354903   
question10    0.945573  0.640922  0.551092  0.321063  0.381911  0.143033   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label                                       ...                             
question1

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
クラス1の確率: [0.5        0.53654589 0.29405973 0.16839493 0.43098654 0.44292901
 0.30654251 0.30918547 0.58254208 0.68360924]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.14150888 0.06747711 0.07418388 0.08463468 0.05697832 0.08293831
 0.08707127 0.13565031 0.10607228 0.04317559]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.05996899 0.03463398 0.03395744 0.04429444 0.06145236 0.02675007
 0.10174253 0.09026542 0.0102113 ]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.27841424 0.13929196 0.17958964 0.15879303 0.18105682 0.0485331

In [12]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.37895772 0.38578488 0.22768417 0.12424884 0.31812549 0.37799702
 0.29511297 0.25624957 0.46597584 0.52489203]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.19402839 0.07604224 0.04039466 0.17341718 0.13982988 0.14593795
 0.10318428 0.12869422 0.17399482 0.08611815]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.14005524 0.06401446 0.07385262 0.08108032 0.11659691 0.07619232
 0.12406841 0.09183279 0.04696074]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.

最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID70.csv
予測率: 44.44%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 1, 1, 1, 1]
クラス1の確率: [0.35052088 0.5        0.54475357 0.25101453 0.12905568 0.71950754
 0.61874082 0.67198712 0.58550136]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID71.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3653296  0.25052238 0.2436996  0.3615224  0.24407548 0.13052541
 0.24665339 0.2442503  0.2068405  0.15892601]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID72.csv
予測率: 22.22%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 1, 1, 0, 0, 1, 1, 1, 1]
クラス1の確率: [0.73099744 0.75956225 0.63125947 0.46366325 0.38435669 0.65888671
 0.68553031 0.65458936 0.73684158]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID73.csv

## SVM（先頭30フレームの標準偏差）

In [13]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001466  0.082011  0.085878  0.133645  0.058596  0.050013   
question2     0.001270  0.081372  0.124629  0.117510  0.064030  0.045970   
question3     0.001708  0.075652  0.092508  0.122632  0.077482  0.030568   
question4     0.001270  0.083811  0.072418  0.098150  0.072119  0.032883   
question5     0.001197  0.071634  0.093603  0.136910  0.053622  0.084010   
question6     0.001881  0.063219  0.086607  0.123548  0.075216  0.048818   
question7     0.002001  0.067252  0.083081  0.136454  0.096351  0.042810   
question8     0.001401  0.068174  0.114510  0.134832  0.034390  0.065531   
question9     0.001172  0.070727  0.085490  0.118371  0.076434  0.032543   
question10    0.001385  0.052577  0.074468  0.165595  0.090610  0.033810   

                AU07      AU09      AU10      AU11  ... 

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.008259  0.076216  0.113029  0.145275  0.059982  0.091380   
question2     0.006369  0.119858  0.139371  0.169864  0.109141  0.154029   
question3     0.009644  0.095937  0.119963  0.198624  0.134143  0.170419   
question4     0.013622  0.077723  0.120661  0.127663  0.128617  0.102578   
question5     0.017211  0.092391  0.135731  0.141226  0.133872  0.111514   
question6     0.013416  0.084853  0.098202  0.144309  0.086509  0.056125   
question7     0.007244  0.184041  0.185995  0.179585  0.095643  0.017587   
question8     0.022125  0.104102  0.125752  0.117953  0.128482  0.136233   
question9     0.023001  0.082246  0.145109  0.141356  0.138747  0.112585   
question10    0.021895  0.087378  0.150234  0.121084  0.139584  0.137724   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label   

In [9]:

from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.26494291 0.19131203 0.43002037 0.23603749 0.12842112 0.16743978
 0.2804757  0.22045968 0.15320963 0.34244097]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.30643728 0.13512077 0.4140211  0.15523587 0.17150046 0.19919779
 0.2170617  0.21680445 0.25650563 0.22321993]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.2210346  0.05741108 0.07709409 0.08406445 0.1254742  0.09354716
 0.07641743 0.10312452 0.08678773]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.18565195 0.34864123 0.30001194 0.14070721 0.31037878 0.138790

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID30.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 1, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.51489363 0.28550266 0.55285616 0.35975017 0.2614744  0.2341864
 0.41029037 0.38459582 0.22422601 0.47632893]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.45108296 0.25193242 0.22163494 0.22115233 0.30036965 0.39815594
 0.3469058  0.39558838 0.32862534 0.40953049]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.32909995 0.07642461 0.11457526 0.1557305  0.12825489 0.16625884
 0.12126004 0.17207251 0.16558217]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.1, 'kern

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.88928674 0.90631739 0.62249028 0.88506715 0.70888689 0.51057515
 0.82379457 0.90051842 0.61739976]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.42622018 0.3324865  0.48821859 0.71907813 0.28474312 0.22809051
 0.23249171 0.20574518 0.39327584 0.27664563]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4298101  0.3845676  0.36855493 0.43698785 0.44797459 0.52124729
 0.51701491 0.51948055 0.33552407]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73.csv
予測率: 100.00%

## SVM（先頭10フレームの標準偏差）

In [3]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # 標準偏差を計算
    std_values_per_label = first_30_frames_per_label.groupby("label").std(numeric_only=True)

    # NaNを含む行を削除
    std_values_per_label = std_values_per_label.dropna(axis=0)
    
    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001121  0.093524  0.112813  0.113783  0.070513  0.058268   
question2     0.001382  0.089263  0.136364  0.145956  0.027679  0.049899   
question3     0.000877  0.048792  0.071889  0.116778  0.077599  0.029961   
question4     0.000845  0.095508  0.052701  0.104434  0.062406  0.040550   
question5     0.001609  0.054508  0.082077  0.167779  0.059787  0.087103   
question6     0.001885  0.068061  0.087083  0.142162  0.062328  0.051092   
question7     0.001011  0.066099  0.094583  0.117106  0.093062  0.055414   
question8     0.000721  0.064097  0.123955  0.131484  0.025264  0.039280   
question9     0.000666  0.086447  0.063176  0.087117  0.092931  0.036451   
question10    0.001114  0.055978  0.052930  0.178878  0.065129  0.033687   

                AU07      AU09      AU10      AU11  ... 

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question2     0.000319  0.057419  0.110677  0.101285  0.064043  0.076580   
question3     0.000238  0.089027  0.095174  0.069641  0.012112  0.027413   
question4     0.000066  0.065625  0.087000  0.083188  0.057769  0.101090   
question6     0.000068  0.121429  0.097981  0.200236  0.111416  0.129538   
question7     0.000215  0.119393  0.126717  0.127009  0.040794  0.039873   
question8     0.000096  0.074961  0.079535  0.153706  0.031724  0.093554   
question9     0.000131  0.082327  0.060016  0.056461  0.005327  0.028172   
question10    0.001251  0.058221  0.134088  0.202426  0.139298  0.113275   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                               ...                       
question2   0.316228  0.103821  0.102156  0.527046  ...  0.127296  0.065599   
qu

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000074  0.054649  0.104145  0.055182  0.023144  0.042208   
question2     0.000493  0.094378  0.082368  0.080634  0.100275  0.068735   
question3     0.000397  0.043745  0.077711  0.038831  0.027168  0.013131   
question4     0.000555  0.051751  0.073959  0.025297  0.016769  0.006033   
question5     0.000610  0.093931  0.066809  0.025865  0.016459  0.006352   
question6     0.000421  0.116303  0.072854  0.048025  0.013232  0.005905   
question7     0.000228  0.043148  0.052549  0.043807  0.095903  0.008275   
question8     0.000642  0.062896  0.126753  0.037764  0.087343  0.010493   
question10    0.000442  0.047296  0.054595  0.045922  0.116587  0.012179   

            AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                           ...                       
question1    

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.007996  0.087572  0.132737  0.146465  0.062704  0.119484   
question2     0.005135  0.132575  0.124925  0.211021  0.076990  0.170071   
question3     0.011776  0.045625  0.109528  0.242119  0.106644  0.079748   
question4     0.019453  0.053513  0.053514  0.079920  0.129792  0.091010   
question5     0.018167  0.080723  0.109186  0.145466  0.118800  0.093223   
question6     0.010932  0.104754  0.067611  0.130933  0.087272  0.035644   
question7     0.006455  0.182245  0.176745  0.202367  0.092513  0.009990   
question8     0.020566  0.100134  0.114468  0.089797  0.135974  0.160679   
question9     0.022744  0.084196  0.144495  0.172806  0.134686  0.119861   
question10    0.016807  0.106081  0.166942  0.123167  0.143464  0.084387   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label   

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000750  0.128132  0.090615  0.131922  0.029593  0.033560   
question2     0.000527  0.084906  0.091184  0.164134  0.032315  0.048872   
question3     0.000578  0.046512  0.084092  0.101249  0.026708  0.063288   
question5     0.000252  0.090117  0.048867  0.137518  0.025333  0.034379   
question6     0.000375  0.107674  0.148931  0.168815  0.042535  0.034729   
question7     0.000372  0.068746  0.052044  0.150345  0.042873  0.029309   
question8     0.001040  0.130267  0.152652  0.107332  0.071224  0.066599   
question9     0.000460  0.136325  0.211371  0.128928  0.027526  0.029646   
question10    0.000446  0.082172  0.095932  0.071236  0.033894  0.049857   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label                                               ...                       
quest

In [4]:

from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.237144   0.1858206  0.64630645 0.17110452 0.38357417 0.3650512
 0.2836456  0.61547606 0.20280553 0.46079997]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.67527323 0.19842313 0.18202114 0.1804676  0.18854634 0.15356676
 0.51637084 0.23320059 0.22089932 0.48171489]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.48833722 0.06907231 0.09126243 0.11027204 0.09033464 0.10257903
 0.09269098 0.11424103 0.07869293]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.24337008 0.27599959 0.22297611 0.21721566 0.33547251 0.14840043


In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.35082329 0.3067516  0.40449588 0.29234176 0.3766991  0.37724935
 0.35530003 0.39911251 0.32347284 0.36594791]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.67242507 0.28683837 0.16643544 0.25947142 0.2591094  0.21667595
 0.5        0.2109948  0.27257241 0.43020325]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.41976179 0.12177758 0.16530945 0.22136336 0.16739972 0.19433404
 0.1530291  0.15936897 0.16698907]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 1, 'gamma': 1, 'kernel': 'rbf

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.68959722 0.81217824 0.57821045 0.80162292 0.40154888 0.39269108
 0.67616593 0.82021994 0.44586255]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.51445621 0.2979367  0.35615141 0.48953391 0.34975435 0.26530601
 0.27670178 0.27420115 0.34938968 0.26132983]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72.csv
予測率: 88.89%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 0, 0]
クラス1の確率: [0.42643018 0.28048556 0.30407647 0.35615873 0.25968023 0.51401171
 0.27103872 0.29303318 0.48010222]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73.csv
予測率: 90.00%


## SVM（先頭30フレームの差分）

In [10]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 結果をリストに保存
    result_list.append((file_name, diff_values_per_label))

    # Yを追加
    diff_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
          label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     question1   -0.001724 -0.251561 -0.248012  0.076567 -0.139892  0.030934   
2     question1    0.000788  0.066084  0.033166 -0.046987  0.005213 -0.070424   
3     question1   -0.000408  0.121311  0.016596  0.155112  0.035684  0.133026   
4     question1    0.000278 -0.113827 -0.060162  0.081835 -0.004229 -0.135589   
5     question1    0.001025  0.059989  0.139242 -0.150123 -0.069870  0.067338   
..          ...         ...       ...       ...       ...       ...       ...   
295  question10   -0.000181  0.001104  0.085981  0.045987 -0.157633  0.066186   
296  question10   -0.001860  0.067276 -0.032094 -0.116374  0.043166 -0.022014   
297  question10    0.002063 -0.006732 -0.072016  0.150383 -0.033546 -0.031211   
298  question10   -0.003481 -0.029945  0.094034 -0.154169  0.187788  0.032302   
299  question10   -0.001027 -0.039412 -0.024116  0.091600 -0.018093 -0.039674 

          label  Confidence      AU01      AU02      AU04      AU05      AU06  \
1     question1   -0.000845 -0.095166  0.040092 -0.035933  0.027117  0.086048   
2     question1    0.000036  0.095684 -0.082437  0.353576 -0.005651 -0.163359   
3     question1    0.000921 -0.021138 -0.004248 -0.135452  0.095453 -0.055541   
4     question1    0.000321  0.333401  0.103104 -0.305854 -0.089187  0.091699   
5     question1   -0.000585 -0.293075 -0.028335  0.505766  0.083529 -0.059175   
..          ...         ...       ...       ...       ...       ...       ...   
295  question10   -0.000163  0.012287  0.046693  0.083211  0.085071  0.035229   
296  question10   -0.002379  0.018447 -0.100269  0.031060 -0.042632  0.008392   
297  question10    0.001601  0.034684  0.070767  0.020186  0.003733 -0.019650   
298  question10    0.000884 -0.014804 -0.025379 -0.001735  0.026950  0.007884   
299  question10   -0.000217 -0.031524  0.032451  0.006000  0.005377  0.005469   

     AU07      AU09      AU

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    #print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 95.17%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.cs

ファイル名: filtered_ID40.csv
予測率: 96.21%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 12/31 =====
ファイル名: filtered_ID41.csv
予測率: 98.85%
正解ラベル: [0, 0, 0,

ファイル名: filtered_ID49.csv
予測率: 49.75%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 21/31 =====
ファイル名: filtered_ID50.csv
予測率: 54.83%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

ファイル名: filtered_ID72.csv
予測率: 82.38%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 31/31 =====
ファイル名: filtered_ID73.csv
予測率: 90.34%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

## SVM（先頭30フレームの差分の標準偏差）

In [15]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )
    
    # ラベルごとにフレーム間の差分を計算
    diff_values_per_label = (
        first_30_frames_per_label.groupby("label")
        .apply(lambda group: group.iloc[:, 1:].diff().dropna())
    )

    # 差分に対応するラベル列をリセット（ラベルを戻す）
    diff_values_per_label = diff_values_per_label.reset_index(level=0).dropna()
    
    # 差分の標準偏差を計算
    std_values_per_label = (
        diff_values_per_label.groupby("label").std(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, std_values_per_label))

    # Yを追加
    std_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001357  0.109357  0.108198  0.137026  0.068213  0.064572   
question2     0.001140  0.100844  0.111314  0.155546  0.080165  0.067363   
question3     0.001540  0.101435  0.111410  0.189246  0.102810  0.040152   
question4     0.001450  0.103023  0.112387  0.138747  0.081161  0.039937   
question5     0.001605  0.113691  0.124395  0.193588  0.075903  0.102474   
question6     0.001716  0.081146  0.126423  0.151933  0.080299  0.062055   
question7     0.002743  0.079252  0.112768  0.150238  0.107974  0.056872   
question8     0.001929  0.095910  0.145074  0.185013  0.050982  0.097036   
question9     0.001081  0.103380  0.134461  0.168435  0.103486  0.049545   
question10    0.001231  0.073412  0.098630  0.219724  0.112046  0.047315   

                AU07      AU09      AU10      AU11  ... 

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000479  0.079053  0.127907  0.095062  0.044867  0.033442   
question2     0.000019  0.032371  0.055438  0.141124  0.053045  0.042876   
question3     0.000019  0.057098  0.071304  0.070259  0.062832  0.022707   
question4     0.000017  0.047637  0.082069  0.138993  0.068845  0.026229   
question5     0.000017  0.060396  0.096145  0.111296  0.011404  0.052811   
question6     0.000037  0.079273  0.136681  0.115619  0.016260  0.037466   
question7     0.000013  0.045283  0.080140  0.062422  0.066462  0.090587   
question8     0.000144  0.078512  0.143361  0.124231  0.027448  0.078008   
question9     0.000029  0.082299  0.092844  0.139986  0.021029  0.065477   
question10    0.000128  0.086172  0.112618  0.137664  0.066723  0.088464   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label   

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.031755  0.214618  0.190248  0.178463  0.089114  0.193056   
question2     0.030496  0.223381  0.269690  0.197830  0.075862  0.138875   
question3     0.009183  0.103818  0.125775  0.128835  0.134638  0.220785   
question4     0.015408  0.170741  0.195138  0.209033  0.114517  0.291632   
question5     0.018004  0.170389  0.208080  0.194385  0.093012  0.214533   
question6     0.011461  0.186473  0.215786  0.205461  0.103197  0.351004   
question7     0.007037  0.199347  0.246693  0.202869  0.113016  0.404257   
question8     0.025993  0.186621  0.172078  0.164613  0.089059  0.337737   
question9     0.009880  0.183977  0.154144  0.191342  0.129948  0.328750   
question10    0.021195  0.135712  0.180034  0.201787  0.135373  0.095921   

                AU07      AU09      AU10      AU11  ...      AU15      AU17  \
label   

In [13]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 62.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 1, 1, 0, 1, 1, 1]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデー

In [16]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID30.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.5        0.47467688 0.70044595 0.42815804 0.3284623  0.23215145
 0.39751371 0.55638508 0.24493795 0.50544505]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.41974935 0.21906541 0.18297057 0.18493619 0.23718619 0.36424002
 0.26538587 0.3583815  0.3230901  0.31666194]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.09294731 0.10664705 0.03946708 0.08551134 0.12299461 0.11209488
 0.09519615 0.08201995 0.18294865]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'ke

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.84471502 0.86578071 0.6776806  0.82759098 0.759767   0.52602608
 0.71071654 0.85695647 0.56055249]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 1, 1, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.42881982 0.65425046 0.7818285  0.95645638 0.13384952 0.25571593
 0.25464111 0.07999438 0.3311002  0.5       ]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72.csv
予測率: 77.78%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 1, 1, 0, 0, 0]
クラス1の確率: [0.20782279 0.24035133 0.21605374 0.55882591 0.62005692 0.77484771
 0.33500315 0.53742827 0.28487622]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73.csv
予測率: 100.00%
正解

## SVM（先頭30フレームの最大値）

In [2]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに30フレームの最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.998051  0.806033  0.480008  0.717420  0.461953  0.308946   
question2     0.998089  0.767921  0.681266  0.755417  0.462867  0.325665   
question3     0.998311  0.795240  0.590224  0.725626  0.549736  0.252454   
question4     0.998152  0.756844  0.466964  0.659100  0.487703  0.237110   
question5     0.997102  0.767689  0.505109  0.819110  0.420315  0.500384   
question6     0.998499  0.725317  0.590334  0.698781  0.572304  0.347129   
question7     0.997608  0.784583  0.525491  0.776084  0.581872  0.351664   
question8     0.998259  0.683937  0.604178  0.745114  0.400974  0.502183   
question9     0.998850  0.825670  0.604052  0.809606  0.493420  0.267617   
question10    0.998399  0.836448  0.547312  0.800809  0.504607  0.276122   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.998465  0.752216  0.729164  0.365759  0.474210  0.290062   
question2     0.999567  0.672086  0.551222  0.812744  0.453967  0.128449   
question3     0.999194  0.809797  0.691582  0.702949  0.632799  0.130296   
question4     0.999936  0.887839  0.840272  0.813058  0.300614  0.163981   
question5     0.999823  0.694080  0.625749  0.647601  0.364277  0.121185   
question6     0.999931  0.793173  0.639209  0.635703  0.507102  0.106989   
question7     0.999885  0.755594  0.582994  0.674104  0.517146  0.105304   
question8          NaN       NaN       NaN       NaN       NaN       NaN   
question9     0.999644  0.633608  0.448553  0.600141  0.338652  0.070586   
question10    0.999719  0.698607  0.634224  0.817651  0.427811  0.162069   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.999070  0.869002  0.746690  0.717798  0.323643  0.227748   
question2     0.999806  0.868381  0.792973  0.906311  0.406712  0.108450   
question3     0.999604  0.851107  0.818678  0.712066  0.382902  0.132010   
question4     0.999671  0.856561  0.795868  0.729240  0.400649  0.125859   
question5     0.999485  0.838785  0.737181  0.736555  0.320592  0.126527   
question6     0.999372  0.839738  0.705372  0.712526  0.328597  0.139829   
question7     0.999567  0.788978  0.595910  0.780799  0.343328  0.082548   
question8     0.999611  0.885441  0.829651  0.745535  0.340220  0.082752   
question9     0.996991  0.933217  0.838096  0.790140  0.349421  0.108643   
question10    0.992222  0.889932  0.835453  0.794051  0.353984  0.123512   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.998099  0.673312  0.713268  0.583982  0.461596  0.357719   
question2     0.993812  0.832356  0.816759  0.816523  0.663894  0.418466   
question3     0.997545  0.841218  0.712157  0.836921  0.664296  0.339147   
question4     0.992315  0.860204  0.792216  0.902245  0.700835  0.167889   
question5     0.986403  0.773070  0.673577  0.946101  0.631642  0.365635   
question6     0.987861  0.756023  0.634976  0.617719  0.723016  0.494392   
question7     0.995677  0.744189  0.603709  0.804588  0.532527  0.780728   
question8     0.996840  0.750494  0.694280  0.790938  0.616667  0.477647   
question9     0.994752  0.760579  0.566819  0.732650  0.587130  0.790447   
question10    0.992076  0.777732  0.835804  0.611866  0.528697  0.548250   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.999457  0.839815  0.760313  0.577054  0.662814  0.198474   
question2     0.999482  0.728678  0.608949  0.576310  0.580894  0.166058   
question3     0.999277  0.702653  0.671366  0.947207  0.543363  0.180109   
question4     0.999250  0.801135  0.753437  0.815374  0.568001  0.222197   
question5     0.998610  0.865742  0.849423  0.655665  0.573558  0.344836   
question6     0.999535  0.758265  0.646780  0.669874  0.627732  0.155767   
question7     0.999640  0.758647  0.773486  0.519745  0.563614  0.184699   
question8     0.999140  0.785360  0.739445  0.583456  0.518110  0.170223   
question9     0.999387  0.740830  0.677066  0.867268  0.569703  0.305078   
question10    0.999585  0.750963  0.668256  0.467498  0.542578  0.153768   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

In [3]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 1, 0, 1, 0]

===== テストデータ:

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID30.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.34855084 0.50740115 0.33389032 0.21783686 0.48921843 0.35717016
 0.37175433 0.57217523 0.36389303 0.41409969]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.47550762 0.2406838  0.22589855 0.23491347 0.33730633 0.23607064
 0.24021078 0.52865348 0.31884343 0.26377081]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.18250934 0.08023027 0.09345268 0.06723842 0.08342411 0.06769743
 0.03542016 0.12654153 0.10728966]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.01, 'k

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.64669203 0.64553374 0.64882532 0.68788758 0.70248439 0.27577124
 0.67267675 0.63031784 0.77690836]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID71.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.34106484 0.25047457 0.28874695 0.3024927  0.28223177 0.22948627
 0.24673136 0.22219487 0.27635124 0.22359221]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
ファイル名: filtered_ID72.csv
予測率: 88.89%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 1, 0, 0]
クラス1の確率: [0.46163201 0.40914619 0.3961426  0.44067653 0.48731927 0.41937569
 0.54317629 0.51179895 0.35212082]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID73.csv
予測率:

## SVM（先頭10フレームの最大値）

In [4]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに10フレームの最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )
    
    # 結果をリストに保存
    result_list.append((file_name, max_values_per_label))

    # Yを追加
    max_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.996113  0.675807  0.480008  0.545058  0.461953  0.308946   
question2     0.996619  0.767921  0.681266  0.755417  0.338528  0.325665   
question3     0.998311  0.744807  0.426928  0.673520  0.474646  0.200453   
question4     0.997968  0.749176  0.402691  0.659100  0.433518  0.232112   
question5     0.997069  0.664431  0.374271  0.819110  0.390757  0.500384   
question6     0.998499  0.712010  0.469338  0.695301  0.572304  0.347129   
question7     0.996357  0.784583  0.514673  0.598215  0.571546  0.351664   
question8     0.998259  0.677315  0.544166  0.745114  0.359447  0.314839   
question9     0.998850  0.786054  0.473068  0.628629  0.493420  0.267617   
question10    0.998399  0.790493  0.485790  0.800809  0.504607  0.251513   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1          NaN       NaN       NaN       NaN       NaN       NaN   
question2     0.999861  0.758733  0.567210  0.392522  0.456576  0.388019   
question3     0.999860  0.744054  0.564723  0.677939  0.292632  0.399651   
question4     0.999892  0.666658  0.348704  0.596552  0.512322  0.241028   
question5     0.999892  0.608647  0.447465  0.315507  0.535867  0.182671   
question6     0.999881  0.768317  0.544927  0.390294  0.533961  0.277689   
question7     0.999197  0.783336  0.314565  0.532594  0.313326  0.580781   
question8     0.999725  0.646843  0.248219  0.560443  0.326429  0.466778   
question9     0.999743  0.697755  0.392359  0.520628  0.410471  0.380022   
question10    0.999752  0.607287  0.205545  0.615392  0.342136  0.411948   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.995284  0.821773  0.801093  0.664591  0.488839  0.375927   
question2     0.996227  0.739314  0.598786  0.861240  0.504450  0.528054   
question3     0.987261  0.648757  0.746906  0.829465  0.746883  0.313861   
question4     0.986941  0.723356  0.716921  0.338545  0.708273  0.355425   
question5     0.977788  0.780905  0.725425  0.685720  0.711221  0.333242   
question6     0.981771  0.783178  0.645688  0.674666  0.749380  0.152159   
question7     0.984532  0.842255  0.808278  0.731339  0.679293  0.099689   
question8     0.977810  0.650419  0.579414  0.370790  0.681633  0.620431   
question9     0.979656  0.727333  0.777656  0.729258  0.709229  0.473217   
question10    0.973683  0.797317  0.745641  0.508247  0.743761  0.308885   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.997494  0.608457  0.473950  0.889190  0.495525  0.213041   
question2     0.998743  0.849627  0.596761  0.543358  0.458659  0.192861   
question3     0.998751  0.827619  0.554876  0.715649  0.345523  0.245520   
question4     0.999271  0.819473  0.497770  0.404272  0.362464  0.366741   
question5     0.998101  0.859462  0.618346  0.534208  0.461118  0.432294   
question6     0.998776  0.890806  0.636970  0.594941  0.363844  0.691367   
question7     0.998709  0.814869  0.633888  0.445485  0.467562  0.203018   
question8     0.997978  0.814805  0.630306  0.585172  0.453146  0.474106   
question9     0.999272  0.822237  0.586160  0.655816  0.341996  0.738182   
question10    0.999404  0.889518  0.641643  0.590579  0.442730  0.528507   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    #print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 12.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID39.csv
予

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID30.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
クラス1の確率: [0.39600039 0.38384047 0.34720834 0.15645918 0.53137268 0.35634456
 0.3599546  0.48493194 0.32847975 0.38131411]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.48853357 0.13668387 0.13650339 0.09912168 0.18624257 0.08826127
 0.31850399 0.26767607 0.24697221 0.18452027]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.25178752 0.05444824 0.10834345 0.0495712  0.04061892 0.06655528
 0.07607623 0.09590411 0.03143963]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.0

最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID70.csv
予測率: 77.78%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 1, 1, 1, 0, 1, 1, 1, 1]
クラス1の確率: [0.48174633 0.71714956 0.66312541 0.64896141 0.30407559 0.52577982
 0.77517235 0.77981514 0.70331798]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID71.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.41778882 0.15134177 0.12756769 0.34890506 0.23832312 0.05995918
 0.17073594 0.14941834 0.23536803 0.08810041]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID72.csv
予測率: 77.78%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 1, 1]
クラス1の確率: [0.39484477 0.3476793  0.46013361 0.43505627 0.36608454 0.39046632
 0.44710439 0.63078146 0.6530593 ]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID73.csv

## SVM（先頭30フレームの最大値と最小値の差分）

In [21]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の30フレームを取得
    first_30_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(30))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_30_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_30_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.005760  0.390331  0.345453  0.500715  0.214697  0.221865   
question2     0.005446  0.281593  0.459288  0.494674  0.225900  0.216869   
question3     0.007495  0.276103  0.377743  0.456334  0.303219  0.155282   
question4     0.006056  0.333635  0.316722  0.382445  0.232637  0.119928   
question5     0.005688  0.258043  0.350116  0.502948  0.186523  0.326944   
question6     0.008136  0.235677  0.383564  0.483693  0.333962  0.187738   
question7     0.009249  0.275935  0.323897  0.608640  0.328619  0.236012   
question8     0.005283  0.257964  0.457597  0.555496  0.163178  0.310530   
question9     0.004680  0.308041  0.403254  0.582350  0.254546  0.109945   
question10    0.005946  0.201322  0.287763  0.667440  0.250951  0.135159   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.000246  0.238176  0.334261  0.251520  0.224076  0.155313   
question2     0.001683  0.355635  0.293905  0.312108  0.314341  0.281674   
question3     0.005695  0.176389  0.349633  0.210499  0.236728  0.044917   
question4     0.007587  0.187906  0.252564  0.208676  0.256991  0.040752   
question5     0.004394  0.326967  0.226775  0.150450  0.086598  0.028537   
question6     0.001237  0.313462  0.293116  0.167245  0.110010  0.025740   
question7     0.001029  0.219880  0.258177  0.327753  0.281799  0.035945   
question8     0.005799  0.181263  0.407126  0.110728  0.272602  0.049522   
question9          NaN       NaN       NaN       NaN       NaN       NaN   
question10    0.001763  0.311790  0.298114  0.183765  0.422373  0.043108   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.075563  0.588043  0.510130  0.532618  0.212398  0.828060   
question2     0.091136  0.490778  0.719659  0.510915  0.237654  0.387890   
question3     0.032072  0.360797  0.389937  0.350052  0.396379  0.769804   
question4     0.052231  0.517304  0.651226  0.592562  0.290756  0.841759   
question5     0.076040  0.517230  0.623848  0.542348  0.255334  0.635742   
question6     0.037954  0.542574  0.593118  0.494937  0.275505  0.772136   
question7     0.017589  0.481276  0.614999  0.679005  0.279049  0.790928   
question8     0.089122  0.506787  0.486653  0.469161  0.242078  0.805707   
question9     0.031296  0.475570  0.481606  0.579185  0.450456  0.778832   
question10    0.088737  0.494683  0.486916  0.626584  0.531460  0.363965   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

In [19]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    #print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID39.csv
予

In [22]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID30.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.33509341 0.35871293 0.34791626 0.23734703 0.27403367 0.32205317
 0.29543022 0.38937491 0.27593698 0.31654039]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.63359286 0.35122554 0.26698775 0.27626184 0.4170609  0.37941025
 0.3207811  0.47224007 0.43306421 0.50521958]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3185747  0.09502618 0.14810322 0.21534978 0.10751445 0.18150967
 0.09204032 0.22529473 0.2400704 ]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 10, 'gamma': 0.01, 

最適パラメータ: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID70.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.77560054 0.82350336 0.65499135 0.79511555 0.70520192 0.46188828
 0.78907824 0.80588225 0.563331  ]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID71.csv
予測率: 70.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.30716367 0.57130357 0.20573512 0.54158743 0.43604379 0.35882554
 0.33203251 0.3537475  0.20186189 0.53051377]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID72.csv
予測率: 77.78%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 1, 0, 1, 0]
クラス1の確率: [0.39024318 0.24669495 0.19406003 0.36072139 0.51187695 0.66413333
 0.51298489 0.64546671 0.1857318 ]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID73.csv
予測率: 80.00%
正解ラベ

## SVM（先頭10フレームの最大値と最小値の差分）

In [23]:
# 処理するフォルダのパス
folder_path = './output_filtered_questions'

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
labels = [0,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,1,1,1,0,1,0,0,0]
i = 0
# 各ファイルに対して処理を適用
result_list = []  # 平均値を保存するリスト

for file_name in csv_files:
    file_path = os.path.join(folder_path, file_name)
    
    # データの読み込み
    df = pd.read_csv(file_path, low_memory=False)
    
    # `question1`から`question10`のみをフィルタリング
    filtered_df = df[df["label"].str.match(r'^question(10|[1-9])$', na=False)]
    
    # Confidenceが0.90以上の行を抽出
    filtered_df = filtered_df[filtered_df["Confidence"] >= 0.90]
    
    # `Confidence`列と`AU`で始まる列を抽出
    columns_to_extract = ["label", "Confidence"] + [col for col in filtered_df.columns if col.startswith("AU")]
    filtered_df = filtered_df[columns_to_extract]
    
    # 順序を保つためのカスタム順序のカテゴリを定義
    filtered_df["label"] = pd.Categorical(
        filtered_df["label"],
        categories=[f"question{i}" for i in range(1, 11)],
        ordered=True
    )
    
    # ラベルごとに最初の10フレームを取得
    first_10_frames_per_label = (
        filtered_df.groupby("label")
        .apply(lambda group: group.head(10))
        .reset_index(drop=True)  # グループ化インデックスをリセット
    )

    # ラベルごとに最大値を計算
    max_values_per_label = (
        first_10_frames_per_label.groupby("label").max(numeric_only=True)
    )

    # ラベルごとに最小値を計算
    min_values_per_label = (
        first_10_frames_per_label.groupby("label").min(numeric_only=True)
    )

    # 最大値と最小値の差分を計算
    range_values_per_label = max_values_per_label - min_values_per_label

    
    # 結果をリストに保存
    result_list.append((file_name, range_values_per_label))

    # Yを追加
    range_values_per_label["Y"] = labels[i]
    i = i + 1
    
# 各ファイルの結果を出力
for file_name, mean_values in result_list:
    print(f"Results for {file_name}:")
    print(mean_values)
    print()

Results for filtered_ID30.csv:
            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.003822  0.260105  0.302495  0.328353  0.208069  0.194241   
question2     0.003976  0.267006  0.360994  0.494674  0.101561  0.178335   
question3     0.002594  0.142312  0.174857  0.404229  0.207329  0.089944   
question4     0.002409  0.257331  0.168647  0.344704  0.178452  0.114930   
question5     0.005654  0.154785  0.214451  0.458863  0.156966  0.278399   
question6     0.005794  0.210950  0.262568  0.480213  0.194533  0.183369   
question7     0.003267  0.174082  0.288023  0.403695  0.301149  0.205485   
question8     0.002394  0.223722  0.365346  0.419062  0.071020  0.123186   
question9     0.002268  0.268426  0.201086  0.270355  0.254546  0.109945   
question10    0.003167  0.155367  0.132764  0.588269  0.223793  0.110550   

            AU07      AU09      AU10  AU11  ...      AU1

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001921  0.061249  0.185214  0.146304  0.210827  0.160146   
question2     0.004726  0.103949  0.132052  0.346698  0.066395  0.078528   
question3     0.000396  0.079574  0.198263  0.259747  0.095103  0.113822   
question4     0.000531  0.154920  0.124545  0.421277  0.093603  0.308017   
question5     0.000428  0.095009  0.121240  0.203009  0.028292  0.250943   
question6     0.000719  0.051313  0.104112  0.263299  0.095610  0.184776   
question7     0.003441  0.129070  0.219390  0.306555  0.084006  0.095001   
question8     0.000532  0.074161  0.204366  0.267450  0.037794  0.218676   
question9     0.000108  0.064696  0.132345  0.371012  0.109286  0.070628   
question10    0.000098  0.079338  0.148716  0.396533  0.022040  0.016120   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

            Confidence      AU01      AU02      AU04      AU05      AU06  \
label                                                                      
question1     0.001932  0.407947  0.103104  0.519930  0.116919  0.242116   
question2     0.018532  0.582376  0.460880  0.761885  0.157772  0.443715   
question3     0.002315  0.500411  0.621066  0.732747  0.205842  0.036996   
question4     0.001311  0.447253  0.631073  0.722881  0.116623  0.077790   
question5     0.001236  0.178801  0.159376  0.635052  0.104676  0.202320   
question6     0.002263  0.565013  0.493559  0.588598  0.134827  0.172643   
question7     0.002256  0.477908  0.632530  0.716974  0.166955  0.211339   
question8     0.001104  0.493728  0.621004  0.727246  0.213295  0.151367   
question9     0.001310  0.485997  0.712438  0.724153  0.139347  0.172756   
question10    0.002029  0.496652  0.539861  0.621869  0.149213  0.017695   

            AU07      AU09      AU10  AU11  ...      AU15      AU17  AU20  \
label     

In [21]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence','label', 'AU07', 'AU11', 'AU20']

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルの初期化
    model = SVC(kernel='rbf', probability=True)# 'rbf'カーネルを使用（非線形データの場合）
    model.fit(X_train_combined, y_train_combined)
    
    # 予測
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")  # 小数点以下2桁で%表示
    print(f"正解ラベル: {y_test.tolist()}")  # 正解ラベルをリストとして表示
    #print(f"予測ラベル: {y_pred.tolist()}")  # 予測ラベルをリストとして表示
    #print(f"クラス1の確率: {y_pred_prob}")  # 各テストデータ点のクラス1の確率
    

# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")


===== テストデータ: ファイル 1/31 =====
ファイル名: filtered_ID30.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 2/31 =====
ファイル名: filtered_ID31.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 3/31 =====
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 4/31 =====
ファイル名: filtered_ID33.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 5/31 =====
ファイル名: filtered_ID34.csv
予測率: 37.50%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1]

===== テストデータ: ファイル 6/31 =====
ファイル名: filtered_ID35.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 7/31 =====
ファイル名: filtered_ID36.csv
予測率: 90.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 8/31 =====
ファイル名: filtered_ID37.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 9/31 =====
ファイル名: filtered_ID38.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]

===== テストデータ: ファイル 10/31 =====
ファイル名: filtered_ID39.csv
予

In [24]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# 削除する列を指定
columns_to_drop = ['Confidence', 'label', 'AU07', 'AU11', 'AU20']

# グリッドサーチ用のパラメータ設定
param_grid = {
    'C': [0.1, 1, 10, 100],         # 正則化パラメータ
    'gamma': [1, 0.1, 0.01, 0.001], # RBFカーネルのgamma
    'kernel': ['rbf']               # カーネルはRBF
}

# 全ファイルごとの精度結果を保存するリスト
accuracy_list = []

# 全てのファイルをテストデータとして順番に使うループ
for i in range(len(result_list)):
    print(f"\n===== テストデータ: ファイル {i+1}/{len(result_list)} =====")
    
    # テストデータとして選ぶファイル
    test_file_name, test_df = result_list[i]
    
    # テストデータを前処理
    test_df_cleaned = test_df.drop(columns=[col for col in columns_to_drop if col in test_df.columns])
    # NaNを含む行を削除
    test_df_cleaned = test_df_cleaned.dropna(axis=0)
    
    y_test = test_df_cleaned['Y']
    X_test = test_df_cleaned.drop(columns=['Y'])
    
    # 学習データをリストに格納
    X_train_list = []
    y_train_list = []
    
    for j, (train_file_name, train_df) in enumerate(result_list):
        if i != j:  # テストデータ以外を学習データにする
            train_df_cleaned = train_df.drop(columns=[col for col in columns_to_drop if col in train_df.columns])
            # NaNを含む行を削除
            train_df_cleaned = train_df_cleaned.dropna(axis=0)
            train_df_cleaned.reset_index(inplace=True, drop=True)
            
            X_train = train_df_cleaned.drop(columns=['Y'])  # 'label'も削除
            y_train = train_df_cleaned['Y']
            
            X_train_list.append(X_train)
            y_train_list.append(y_train)
    
    # 学習データを結合
    X_train_combined = pd.concat(X_train_list, ignore_index=True)
    y_train_combined = pd.concat(y_train_list, ignore_index=True)
    
    # SVMモデルのグリッドサーチ
    model = SVC(probability=True)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, n_jobs=-1)
    grid_search.fit(X_train_combined, y_train_combined)
    
    # 最適なモデルを取得
    best_model = grid_search.best_estimator_
    print(f"最適パラメータ: {grid_search.best_params_}")
    
    # テストデータで予測
    y_pred = best_model.predict(X_test)
    y_pred_prob = best_model.predict_proba(X_test)[:, 1]  # クラス1の確率
    
    # 精度計算
    accuracy = accuracy_score(y_test, y_pred) * 100  # 正答率を%に変換
    
    # 精度をリストに保存
    accuracy_list.append((test_file_name, accuracy))
    
    # 結果表示
    print(f"ファイル名: {test_file_name}")
    print(f"予測率: {accuracy:.2f}%")
    print(f"正解ラベル: {y_test.tolist()}")
    print(f"予測ラベル: {y_pred.tolist()}")
    print(f"クラス1の確率: {y_pred_prob}")
    
# 全ファイルごとの精度を表示
print("\n===== 全ファイルごとの予測率 =====")
for file_name, acc in accuracy_list:
    print(f"ファイル名: {file_name}, 予測率: {acc:.2f}%")

# 予測率が50%以上のものをカウント
count_above_50 = sum(1 for _, acc in accuracy_list if acc >= 50)

# 結果を表示
total_files = len(accuracy_list)
print(f"\n===== 精度50%以上のファイル数 =====")
print(f"{total_files}ファイル中 {count_above_50}ファイルが予測率50%以上")

# 正解ラベルが1のもののうち、予測率が50%以上のものをカウント
count_above_50_for_label_1 = sum(
    1 for (_, acc), (test_file_name, test_df) in zip(accuracy_list, result_list)
    if acc >= 50 and (test_df['Y'] == 1).any()
)

# 結果を表示
print(f"\n===== 正解ラベルが1のもののうち予測率50%以上のファイル数 =====")
print(f"正解ラベルが1のファイル中 {count_above_50_for_label_1}ファイルが予測率50%以上")



===== テストデータ: ファイル 1/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID30.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 1, 0, 0, 0, 0, 0, 0, 1]
クラス1の確率: [0.44630966 0.25830565 0.57966865 0.30890775 0.44145963 0.42066137
 0.39174632 0.46124289 0.32737466 0.53701483]

===== テストデータ: ファイル 2/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
ファイル名: filtered_ID31.csv
予測率: 80.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
クラス1の確率: [0.65804238 0.29831209 0.187966   0.31682748 0.33006224 0.2270844
 0.49408463 0.28981227 0.2923833  0.36700977]

===== テストデータ: ファイル 3/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID32.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.3570441  0.13842156 0.16199773 0.24012855 0.20684922 0.16450492
 0.12698433 0.22032317 0.16623721]

===== テストデータ: ファイル 4/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.01, 

最適パラメータ: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
ファイル名: filtered_ID70.csv
予測率: 0.00%
正解ラベル: [1, 1, 1, 1, 1, 1, 1, 1, 1]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.4812727  0.50913995 0.5203292  0.57380579 0.4818683  0.47489062
 0.4796464  0.5        0.47097787]

===== テストデータ: ファイル 29/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID71.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.55966363 0.27757449 0.33774415 0.5        0.33772312 0.2494778
 0.26596849 0.2589678  0.37240263 0.23159757]

===== テストデータ: ファイル 30/31 =====
最適パラメータ: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID72.csv
予測率: 100.00%
正解ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
予測ラベル: [0, 0, 0, 0, 0, 0, 0, 0, 0]
クラス1の確率: [0.43812201 0.40865403 0.35091422 0.43148998 0.39313877 0.38536611
 0.4023482  0.38583791 0.39388686]

===== テストデータ: ファイル 31/31 =====
最適パラメータ: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
ファイル名: filtered_ID73.csv
予測率: 7